In [0]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')
# %mkdir /content/drive/'My Drive'/micronet
%cd  /content/drive/'My Drive'/micronet

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/micronet


In [0]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
from torch.autograd import Variable

import sys
import numpy as np

def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=True)

def conv_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        init.xavier_uniform(m.weight, gain=np.sqrt(2))
        init.constant_(m.bias, 0)
    elif classname.find('BatchNorm') != -1:
        init.constant_(m.weight, 1)
        init.constant_(m.bias, 0)

class wide_basic(nn.Module):
    def __init__(self, in_planes, planes, dropout_rate, stride=1):
        super(wide_basic, self).__init__()
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, padding=1, bias=True)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=True)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=True),
            )

    def forward(self, x):
        out = self.dropout(self.conv1(F.relu(self.bn1(x))))
        out = self.conv2(F.relu(self.bn2(out)))
        out += self.shortcut(x)

        return out

class Wide_ResNet(nn.Module):
    def __init__(self, depth, widen_factor, dropout_rate, num_classes):
        super(Wide_ResNet, self).__init__()
        self.in_planes = 16

        assert ((depth-4)%6 ==0), 'Wide-resnet depth should be 6n+4'
        n = (depth-4)/6
        k = widen_factor

        print('| Wide-Resnet %dx%d' %(depth, k))
        nStages = [16, 16*k, 32*k, 64*k]

        self.conv1 = conv3x3(3,nStages[0])
        self.layer1 = self._wide_layer(wide_basic, nStages[1], n, dropout_rate, stride=1)
        self.layer2 = self._wide_layer(wide_basic, nStages[2], n, dropout_rate, stride=2)
        self.layer3 = self._wide_layer(wide_basic, nStages[3], n, dropout_rate, stride=2)
        self.bn1 = nn.BatchNorm2d(nStages[3], momentum=0.9)
        self.linear = nn.Linear(nStages[3], num_classes)

    def _wide_layer(self, block, planes, num_blocks, dropout_rate, stride):
        strides = [stride] + [1]*(int(num_blocks)-1)
        layers = []

        for stride in strides:
            layers.append(block(self.in_planes, planes, dropout_rate, stride))
            self.in_planes = planes

        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.relu(self.bn1(out))
        out = F.avg_pool2d(out, 8)
        out = out.view(out.size(0), -1)
        out = self.linear(out)

        return out

# if __name__ == '__main__':
#     net=Wide_ResNet(28, 10, 0.3, 10)
#     y = net(Variable(torch.randn(1,3,32,32)))

#     print(y.size())

In [0]:
############### Pytorch CIFAR configuration file ###############
import math

start_epoch = 1
num_epochs = 200
batch_size = 128
optim_type = 'SGD'

mean = {
    'cifar10': (0.4914, 0.4822, 0.4465),
    'cifar100': (0.5071, 0.4867, 0.4408),
}

std = {
    'cifar10': (0.2023, 0.1994, 0.2010),
    'cifar100': (0.2675, 0.2565, 0.2761),
}

# Only for cifar-10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def learning_rate(init, epoch):
    optim_factor = 0
    if(epoch > 89):
        optim_factor = 3
    elif(epoch > 76): # prev it was 120
        optim_factor = 2
    elif(epoch > 60):
        optim_factor = 1

    return init*math.pow(0.2, optim_factor)

def get_hms(seconds):
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)

    return h, m, s

In [0]:
  
from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import sys
import time
import argparse
import datetime


from torch.autograd import Variable

lr = 0.1
net_type = "wide-resnet"
depth = 28
widen_factor = 10
dropout = 0.3
dataset = "cifar100"
resume = True 
testOnly = False

# Hyper Parameter settings
use_cuda = torch.cuda.is_available()
best_acc = 0

# Data Uplaod
print('\n[Phase 1] : Data Preparation')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean[dataset], std[dataset]),
]) # meanstd transformation

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean[dataset], std[dataset]),
])


print("| Preparing CIFAR-100 dataset...")
sys.stdout.write("| ")
trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=False, transform=transform_test)
num_classes = 100

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Return network & file name
def getNetwork():
    net = Wide_ResNet(depth, widen_factor, dropout, num_classes)
    file_name = 'wide-resnet-'+str(depth)+'x'+str(widen_factor)
    return net, file_name

# Test only option
def test():
    print('\n[Test Phase] : Model setup')
    assert os.path.isdir('checkpoint'), 'Error: No checkpoint directory found!'
    _, file_name = getNetwork()
    checkpoint = torch.load('./checkpoint/'+dataset+os.sep+file_name+'.t7')
    net = checkpoint['net']

    if use_cuda:
        net.cuda()
        net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
        cudnn.benchmark = True

    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
      for batch_idx, (inputs, targets) in enumerate(testloader):
          if use_cuda:
              inputs, targets = inputs.cuda(), targets.cuda()
          outputs = net(inputs)

          _, predicted = torch.max(outputs.data, 1)
          total += targets.size(0)
          correct += predicted.eq(targets.data).cpu().sum()

      acc = 100.*correct/total
      print("| Test Result\tAcc@1: %.2f%%" %(acc))


# Model
print('\n[Phase 2] : Model setup')
if resume:
    # Load checkpoint
    print('| Resuming from checkpoint...')
    assert os.path.isdir('checkpoint'), 'Error: No checkpoint directory found!'
    _, file_name = getNetwork()
    checkpoint = torch.load('./checkpoint/'+dataset+os.sep+file_name+'.t7')
    net = checkpoint['net']
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']
else:
    print('| Building net type [' + net_type + ']...')
    net, file_name = getNetwork()
    net.apply(conv_init)

if use_cuda:
    net.cuda()
    net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()

# Training
def train(epoch):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    optimizer = optim.SGD(net.parameters(), lr=learning_rate(lr, epoch), momentum=0.9, weight_decay=5e-4)

    print('\n=> Training Epoch #%d, LR=%.4f' %(epoch, learning_rate(lr, epoch)))
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda() # GPU settings
        optimizer.zero_grad()
        outputs = net(inputs)               # Forward Propagation
        loss = criterion(outputs, targets)  # Loss
        loss.backward()  # Backward Propagation
        optimizer.step() # Optimizer update

        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

        sys.stdout.write('\r')
        sys.stdout.write('| Epoch [%3d/%3d] Iter[%3d/%3d]\t\tLoss: %.4f Acc@1: %.3f%%'
                %(epoch, num_epochs, batch_idx+1,
                    (len(trainset)//batch_size)+1, loss.item(), 100.*correct/total))
        sys.stdout.flush()

def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
      for batch_idx, (inputs, targets) in enumerate(testloader):
          if use_cuda:
              inputs, targets = inputs.cuda(), targets.cuda()
          outputs = net(inputs)
          loss = criterion(outputs, targets)

          test_loss += loss.item()
          _, predicted = torch.max(outputs.data, 1)
          total += targets.size(0)
          correct += predicted.eq(targets.data).cpu().sum()

      # Save checkpoint when best model
      acc = 100.*correct/total
      print("\n| Validation Epoch #%d\t\t\tLoss: %.4f Acc@1: %.2f%%" %(epoch, loss.item(), acc))

      # if acc > best_acc:
      print('| Saving Best model...\t\t\tTop1 = %.2f%%' %(acc))
      state = {
              'net':net.module if use_cuda else net,
              'acc':acc,
              'epoch':epoch,
      }
      if not os.path.isdir('checkpoint'):
          os.mkdir('checkpoint')
      save_point = './checkpoint/'+dataset+os.sep
      if not os.path.isdir(save_point):
          os.mkdir(save_point)
      torch.save(state, save_point+file_name+'.t7')
      best_acc = acc

print('\n[Phase 3] : Training model')
print('| Training Epochs = ' + str(num_epochs))
print('| Initial Learning Rate = ' + str(lr))
print('| Optimizer = ' + str(optim_type))

elapsed_time = 0
for epoch in range(start_epoch, start_epoch+num_epochs):
    start_time = time.time()

    train(epoch)
    test(epoch)

    epoch_time = time.time() - start_time
    elapsed_time += epoch_time
    print('| Elapsed time : %d:%02d:%02d'  %(get_hms(elapsed_time)))

print('\n[Phase 4] : Testing model')
print('* Test results : Acc@1 = %.2f%%' %(best_acc))


[Phase 1] : Data Preparation
| Preparing CIFAR-100 dataset...
| Files already downloaded and verified

[Phase 2] : Model setup
| Resuming from checkpoint...
| Wide-Resnet 28x10

[Phase 3] : Training model
| Training Epochs = 200
| Initial Learning Rate = 0.1
| Optimizer = SGD

=> Training Epoch #88, LR=0.0040
| Epoch [ 88/200] Iter[391/391]		Loss: 0.0131 Acc@1: 99.000%
| Validation Epoch #88			Loss: 0.8938 Acc@1: 78.00%
| Saving Best model...			Top1 = 78.00%


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Wide_ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type wide_basic. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


| Elapsed time : 0:04:56

=> Training Epoch #89, LR=0.0040
| Epoch [ 89/200] Iter[391/391]		Loss: 0.0370 Acc@1: 99.000%
| Validation Epoch #89			Loss: 0.9738 Acc@1: 78.00%
| Saving Best model...			Top1 = 78.00%
| Elapsed time : 0:09:52

=> Training Epoch #90, LR=0.0008
| Epoch [ 90/200] Iter[391/391]		Loss: 0.0169 Acc@1: 99.000%
| Validation Epoch #90			Loss: 1.0305 Acc@1: 78.00%
| Saving Best model...			Top1 = 78.00%
| Elapsed time : 0:14:48

=> Training Epoch #91, LR=0.0008
| Epoch [ 91/200] Iter[391/391]		Loss: 0.0106 Acc@1: 99.000%
| Validation Epoch #91			Loss: 0.9206 Acc@1: 79.00%
| Saving Best model...			Top1 = 79.00%
| Elapsed time : 0:19:44

=> Training Epoch #92, LR=0.0008
| Epoch [ 92/200] Iter[391/391]		Loss: 0.0160 Acc@1: 99.000%
| Validation Epoch #92			Loss: 0.9434 Acc@1: 79.00%
| Saving Best model...			Top1 = 79.00%
| Elapsed time : 0:24:41

=> Training Epoch #93, LR=0.0008
| Epoch [ 93/200] Iter[391/391]		Loss: 0.0110 Acc@1: 99.000%
| Validation Epoch #93			Loss: 0.983